In [1]:
files = ['/Users/alextaim/data/ccb/original/01.json']

In [2]:
from VwPipeline import Loggers, Handlers
from VwPipeline.VwCache import VwCache
from VwPipeline.Vw import Vw
from VwPipeline.VwOpts import dimension, product
import pandas as pd

#your vw path
vw_path = r'vw'

cache = VwCache(r'/Users/alextaim/data/.vw_cache')
vw = Vw(
    vw_path,
    cache,
    handlers=[Handlers.WidgetHandler()],
    )

opts = pd.DataFrame(product(
    dimension('#base', ['--ccb_explore_adf --dsjson --compressed --synthcover --power_t 0  -P 1 --preserve_performance_counters --save_resume']),
))
preds = vw.train(files, opts, ['-p'])
prediction_files = preds.iloc[0]['!Outputs']['-p']
prediction_files

['/Users/alextaim/data/.vw_cache/cache-p/f70c85ace19b22cd7f9b7d447761ecf0']

In [3]:
def ccb_predictions(files, index = 't'):
    for f in files:
        for l in open(f):


class CcbPredictions:
    def __init__(self, files, index='t'):
        self.files = files
        self.index = index
        
    def iter_slots(self):
        result = {}
        for f in files:
            for l in open(f):
                o = json.loads(l)
                for 
                    yield result

IndentationError: expected an indented block (<ipython-input-3-18f64898fe11>, line 6)

# Usage

## Generate slim dsjson

In [ ]:
from Pipeline.ccb.dsjson import processor

In [5]:
from Pipeline.dataflow import FilesPipeline
from Pipeline.ccb.dsjson.processor import Processor

processor = Processor()
pipeline = FilesPipeline()

slim = pipeline.lines_2_lines(files, processor.process, path_gen=lambda p: p.replace('original', 'slim'), process=True)
slim

['/Users/alextaim/data/ccb/slim/01.json']

## Sample

In [6]:
from Pipeline.dataflow import FilesPipeline
from Pipeline.ccb.dsjson.processor import Processor
from Pipeline.filters import UniformSampler

sampler = UniformSampler(0.5)

processor = Processor(filters=[lambda l: sampler.do(l)])
result = pipeline.lines_2_lines(files, processor.process, path_gen=lambda p: p.replace('original', 'sample'), process=True)
result

['/Users/alextaim/data/ccb/sample/01.json']

## Predict

In [7]:
from Pipeline.dataflow import FilesPipeline
from Pipeline.ccb.dsjson.predictions import Predictor

predictor = Predictor(filters=[lambda l: True])
pipeline = FilesPipeline()
cfe = pipeline.lines_2_lines(slim, predictor.predict, path_gen=lambda p: p.replace('slim', 'predict'), process=True)
cfe

['/Users/alextaim/data/ccb/predict/01.json']

# Preestimate

In [20]:
from Pipeline.dataflow import FilesPipeline
from Pipeline.estimators import Estimator, evaluate
import Pipeline.ccb.estimators
import json

pipeline = FilesPipeline()
estimator = Estimator(factory = Pipeline.ccb.estimators.create, estimators = {'baseline1_old': ['ccb|snips|0'], 'random': ['ccb|snips|0']}, online_estimator = 'ccb|ips|0')
preestimates = pipeline.ndjson_2_csv(cfe, lambda objects: estimator.preestimate(objects, '1min'), path_gen=lambda p: p.replace('predict', 'estimate'), process=True)


# Evaluate

In [21]:
baseline_preestimates = pd.concat([estimator.read_preestimate(p) for p in preestimates])
baseline_stats = evaluate(baseline_preestimates.resample('1min').sum())
baseline_stats

,"(online, ccb|ips|0, e)","(baseline1_old, ccb|snips|0, e)","(random, ccb|snips|0, e)"
t,,,
2021-03-27 17:09:00+00:00,0.333333,0.333333,0.319149
2021-03-27 17:10:00+00:00,0.090909,0.090909,0.088975
2021-03-27 17:11:00+00:00,0.091787,0.071429,0.090767
2021-03-27 17:12:00+00:00,0.099622,0.141935,0.084033
2021-03-27 17:13:00+00:00,0.103704,0.086580,0.082170


# Evaluate predictions from vw

In [22]:
from Pipeline.ccb.vw import predictions
vw_pred_openers = [lambda f: predictions.lines_2_predictions(open(p), open(f), 'cfe') for p in prediction_files]

In [23]:
pipeline = FilesPipeline()
estimator = Estimator(factory = Pipeline.ccb.estimators.create, estimators = {'cfe': ['ccb|snips|0'] })
preestimates = pipeline.ndjson_2_csv(cfe, lambda objects: estimator.preestimate(objects, '1min'), path_gen=lambda p: p.replace('predict', 'cfe'), process=True, openers=vw_pred_openers)


In [24]:
baseline_preestimates = pd.concat([estimator.read_preestimate(p) for p in preestimates])
cfe_stats = evaluate(baseline_preestimates.resample('1min').sum())
cfe_stats

,"(cfe, ccb|snips|0, e)"
t,
2021-03-27 17:09:00+00:00,0.320513
2021-03-27 17:10:00+00:00,0.108569
2021-03-27 17:11:00+00:00,0.086635
2021-03-27 17:12:00+00:00,0.131921
2021-03-27 17:13:00+00:00,0.161565


In [25]:
all_stats = baseline_stats.join(cfe_stats)

,"(online, ccb|ips|0, e)","(baseline1_old, ccb|snips|0, e)","(random, ccb|snips|0, e)","(cfe, ccb|snips|0, e)"
t,,,,
2021-03-27 17:09:00+00:00,0.333333,0.333333,0.319149,0.320513
2021-03-27 17:10:00+00:00,0.090909,0.090909,0.088975,0.108569
2021-03-27 17:11:00+00:00,0.091787,0.071429,0.090767,0.086635
2021-03-27 17:12:00+00:00,0.099622,0.141935,0.084033,0.131921
2021-03-27 17:13:00+00:00,0.103704,0.086580,0.082170,0.161565
